In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
print(tf.__version__)

2.6.1


In [2]:
#image  module rotate or convert tif in jpg
import os
import json
import cv2
import PIL 
from PIL import Image 



class image_converter():
    def __init__(self, abs_path = ''):
        if abs_path:
            self.main_path = abs_path
        else:
            self.main_path = os.path.join(os.getcwd(), 'data')
        
        self.dir_json = list_dir_fls(folder=self.main_path)
    
    
        
    def list_dir_fls(self, folder='data', current_folder = True):
        """return path dictionary"""
        answer = {'folder': '', 'subfolders': [], 'files': []}
        try:
            if current_folder:
                data_path = os.path.join(prog_path, folder)
            else:
                data_path = folder
            answer['folder'] = data_path
            folders = [os.path.join(data_path, name) for name in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, name))]
            files = [os.path.join(data_path, name) for name in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, name))]
            for fold in folders:
                answer['subfolders'].append(list_dir_fls(folder=fold, current_folder = False))
            answer['files'] = files
        except Exception as e:
            print('Error in module list_dir', e)

#         with open('folders_tree.json', 'w') as outfile:
#             json.dump(answer, outfile)
        return(answer)

    def rotate_files(self):
        """rotate file 3 times clockwise 90 degree and write as _90 _180 _270"""
        files_list = self.dir_json['files']
        print(files_list)
        for file in files_list:
            splited_tag = file.split('/')
            file_folder = '/'.join(splited_tag[0:-1])
            file_name = splited_tag[-1]
            file_abs_name = file_name.split('.')[0:-1]
            file_abs_name = '.'.join(file_abs_name)
            file_extention = file_name.split('.')[-1]
            if file_extention == 'jpg':
                for degree in [90, 180,270]:
                    pil_image = Image.open(file)
                    rotated = pil_image.rotate(degree, expand=True)
                    file_name_new = f'{file_abs_name}_{degree}.{file_extention}'
                    file_save = rotated.save(f'{file_folder}/{file_name_new}')
                    print(file_name_new)
        pass

    def rename_tif_files(self):
        """ rename and convert tif ro jpg in directory"""
        list_folders_files = self.dir_json['files']
        main_folder_path = self.dir_json['folder']
        self.make_jpg_dir()
        for sub_f in self.dir_json['subfolders']:
            num = 0
            folder_path = sub_f['folder']
            folder_name = folder_path.split('/')[-1]
            for file_path in sub_f['files']:
                abs_name = file_path.split('/')[-1]
                extention = str(abs_name).split('.')[-1]
                if extention == 'tif':
                    new_abs_name = f'{folder_name}.{num}.jpg'
                    new_name = f'{self.main_path}/jpg/{new_abs_name}'
                    read = cv2.imread(file_path)
                    cv2.imwrite(new_name,read,[int(cv2.IMWRITE_JPEG_QUALITY), 100])   
                    # os.rename(file_path, new_name) #it just rename -not convert
                    num+=1
                else:
                    continue
    
    def make_jpg_dir(self):
        try:
            os.mkdir(f'{self.main_path}/jpg')
        except Exception as e:
            print(f'folder {self.main_path} already exist', e)
                
# def del_file(folder_path='data/marked', ext='jpg'):
#   """clear files with extentions """
#   list_folders_files = list_dir_fls(folder=folder_path)
#   for sub_f in list_folders_files['subfolders']:
#     num = 0
#     folder_path = sub_f['folder']
#     folder_abs_name = sub_f['folder'].split('/')[-1]
#     for file_path in sub_f['files']:
#       abs_name = file_path.split('/')[-1]
#       extention = abs_name.split('.')
#       if extention == ext:
#         os.remove(file_path)

#convertion = image_converter('/Users/johnlennon/My Drive/Python/Pdf/data/label_annotated/upd')
#convertion = image_converter('/Users/johnlennon/My Drive/Python/Pdf/data/label_annotated/invoice')
#convertion.rename_tif_files()
#convertion.rotate_files()

In [3]:
# module for convert xml to csv after labelim annotations
import xml.etree.ElementTree as ET
import glob
import pandas as pd

def xml_to_csv(path):
    """Iterates through all .xml files (generated by labelImg) in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """

    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    dir_name = path.split('/')[-1]
    xml_df.to_csv(f'{path}/{dir_name}.csv', sep='\t', encoding='utf-8', index=False)
    print(f'Successfully converted .xml to .csv in directory {dir_name}')
    return xml_df

print(xml_to_csv('/Users/johnlennon/My Drive/Python/Pdf/data/label_annotated_train/test'))


FileNotFoundError: [Errno 2] No such file or directory: '/Users/johnlennon/My Drive/Python/Pdf/data/label_annotated_train/test/test.csv'

In [ ]:
#create tensorflow records






def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [ ]:
file_name = 'sf_serman.pdf'
file_name.split('.')

In [ ]:
for i in [1,2,3]:
    print(i)